In [2]:
%run 01_setup.ipynb

23/11/19 18:51:58 WARN SSLSocketFactoryEx: Failed to load OpenSSL. Falling back to the JSSE default.


In [3]:
import time
from pyspark.sql.functions import to_date, sequence, explode, sum, month, year, concat_ws, collect_list,desc

In [4]:
# take only clear data
ex_clear = ex_df.filter(to_date("srch_ci") <= to_date("srch_co"))

In [5]:
#sum booked rooms by dates
ex_df1 = ex_clear.select('hotel_id', explode(sequence(to_date('srch_ci'),to_date('srch_co'))).alias('date'), 'srch_rm_cnt')
visits_df = ex_df1.groupBy('hotel_id', 'date') \
    .agg(concat_ws("_", year('date'), month('date')).alias('year_month'),sum('srch_rm_cnt').alias('used_rooms'))

In [14]:
visits_df.explain(mode="formatted")

== Physical Plan ==
AdaptiveSparkPlan (10)
+- HashAggregate (9)
   +- HashAggregate (8)
      +- Project (7)
         +- Generate (6)
            +- Filter (5)
               +- GlobalLimit (4)
                  +- Exchange (3)
                     +- LocalLimit (2)
                        +- Scan avro  (1)


(1) Scan avro 
Output [4]: [srch_ci#40, srch_co#41, srch_rm_cnt#44, hotel_id#47L]
Batched: false
Location: InMemoryFileIndex [abfss://data@styakovdwesteurope.dfs.core.windows.net/expedia]
ReadSchema: struct<srch_ci:string,srch_co:string,srch_rm_cnt:int,hotel_id:bigint>

(2) LocalLimit
Input [4]: [srch_ci#40, srch_co#41, srch_rm_cnt#44, hotel_id#47L]
Arguments: 1000

(3) Exchange
Input [4]: [srch_ci#40, srch_co#41, srch_rm_cnt#44, hotel_id#47L]
Arguments: SinglePartition, ENSURE_REQUIREMENTS, [plan_id=37]

(4) GlobalLimit
Input [4]: [srch_ci#40, srch_co#41, srch_rm_cnt#44, hotel_id#47L]
Arguments: 1000, 0

(5) Filter
Input [4]: [srch_ci#40, srch_co#41, srch_rm_cnt#44, hotel_id#47L]

In [11]:
# itterate though distinct year_month periods, collect top 10 hotels in every period
start_time = time.time()

list_of_rows = []
distinct_yms = [x.year_month for x in visits_df.select('year_month').distinct().collect()]
for ym in distinct_yms:
    list_of_rows.extend(visits_df.filter(visits_df.year_month == ym).orderBy(desc("used_rooms")).take(10))
top10_by_month = spark.createDataFrame(list_of_rows)

print("execution time is ", round(time.time() - start_time, 1), "sec.")

execution time is  40.80367469787598


In [15]:
top10_by_month.explain(mode="formatted")

== Physical Plan ==
* Scan ExistingRDD (1)


(1) Scan ExistingRDD [codegen id : 1]
Output [4]: [hotel_id#140L, date#141, year_month#142, used_rooms#143L]
Arguments: [hotel_id#140L, date#141, year_month#142, used_rooms#143L], MapPartitionsRDD[105] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:0, ExistingRDD, UnknownPartitioning(0)




In [59]:
top10_by_month.show()

+-------------+----------+----------+----------+
|     hotel_id|      date|year_month|used_rooms|
+-------------+----------+----------+----------+
|3384434229251|2017-08-24|    2017_8|         4|
|3384434229251|2017-08-19|    2017_8|         4|
|3384434229251|2017-08-20|    2017_8|         4|
|3384434229251|2017-08-21|    2017_8|         4|
|3384434229251|2017-08-22|    2017_8|         4|
|3384434229251|2017-08-23|    2017_8|         4|
|1889785610243|2017-08-17|    2017_8|         3|
|  94489280513|2017-08-26|    2017_8|         3|
|1889785610243|2017-08-16|    2017_8|         3|
|  94489280513|2017-08-25|    2017_8|         3|
|1889785610247|2017-09-10|    2017_9|         7|
|1889785610247|2017-09-11|    2017_9|         7|
|1889785610247|2017-09-12|    2017_9|         7|
|1443109011462|2017-09-02|    2017_9|         4|
|1443109011462|2017-09-01|    2017_9|         4|
|1443109011462|2017-09-03|    2017_9|         4|
|1443109011462|2017-09-04|    2017_9|         4|
|1872605741056|2017-